In [1]:
import pandas as pd

data = pd.read_csv('diabetes.csv')
Y = data['class']
X = data.drop(['class'],axis=1)

In [2]:
from sklearn import tree
from sklearn.model_selection import train_test_split

clf = tree.DecisionTreeClassifier(criterion = 'entropy',
min_samples_leaf = 15)

In [3]:
x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size=0.33)
print(x_train.size)
print(y_train.size)
clf = clf.fit(x_train, y_train)

4112
514


In [4]:
y_predict = clf.predict(x_test)

In [5]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_predict)
acc

0.7401574803149606